# Banking Loan Informations

Banking information and data reformatting 

## Goal

I’ve been tasked with collaborating with a bank to clean data from a recent marketing campaign. It aimed to encourage customers to take out personal loans. The bank plans to run more campaigns in the future and wants me to ensure the data adheres to their specified structure and data types. This will allow them to use the cleaned data I provide to create a PostgreSQL database, which will store the current campaign’s data and enable easy importation of data from future campaigns.

## Setup

Importing the necessary libraries for the project

In [ ]:
import pandas as pd
import numpy as np

### Reading in Data and subsetting into three dataframes

In [ ]:
marketing = pd.read_csv("bank_marketing.csv")

# Subset the entire data file into three different tables for better visualization 'client', 'campaign','economics'
client = marketing[["client_id", "age", "job", "marital", 
                    "education", "credit_default", "mortgage"]]
campaign = marketing[["client_id", "number_contacts", "month", "day", 
               "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
economics = marketing[["client_id", "cons_price_idx", "euribor_three_months"]]

Editing Client Dataset

In [ ]:
## Editing the client dataset
# Clean education column
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

# Clean job column
client["job"] = client["job"].str.replace(".", "_")

# Converting columns into desired data types
for col in ["credit_default", "mortgage"]:
  client[col] = client[col].map({"yes": 1,
                                 "no": 0,
                                 "unknown": 0})
  client[col] = client[col].astype(bool)

Editing campaign Dataset

In [ ]:
# Editing the campaign dataset
# Change campaign_outcome to binary values
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

# Convert previous_outcome to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, 
                                                                 "failure": 0,
                                                                 "nonexistent": 0})

# Add year column
campaign["year"] = "2022"

# Convert day to string data type
campaign["day"] = campaign["day"].astype(str)

# Add last_contact_date column
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

# Convert to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")

# Clean and convert outcome columns to bool
for col in ["campaign_outcome", "previous_outcome"]:
  campaign[col] = campaign[col].astype(bool)

# Drop unneccessary columns
campaign.drop(columns=["month", "day", "year"], inplace=True)

Finally, save the cleaned tables to csv files

In [ ]:
# Save tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)